# Intro to Quantum Computing with Python and Q\# <br> [//build 2019](https://mybuild.techcommunity.microsoft.com/sessions/77163) #

Let's use Python interoperability for the [Quantum Development Kit](https://github.com/Microsoft/Quantum) to introduce users to Q\# and quantum computing more generally.

---


Installation instructions for running this notebook on your machine can be found [here](https://docs.microsoft.com/en-ca/quantum/install-guide/python?view=qsharp-preview).

Let's get started by loading the Python package for Q# interoperability called `qsharp`.

In [ ]:
import qsharp
qsharp.component_versions()

## What is a qubit?

A single qubit can be represented by a vector of 2 complex numbers* like this:

$\left|{x}\right\rangle = \left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

<small>* some conditions apply</small>

## What can we _do_ with a qubit?

You can do three types of things with qubits:

- Prepare a qubit
- Do operations with a qubit
- Measure a qubit : returns a 0 or 1

### Preparing a qubit

In [ ]:
prepare_qubit = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation PrepareQubit() : Unit {
    using (qubit = Qubit()) {     // We want 1 qubit to use for our task
            DumpMachine();         // Print out what the simulator is keeping a record of
    }
}
""")

In [ ]:
prepare_qubit.simulate()

You can read the above output like the vector we wrote above, where the first column is the index, the second is the real part of the vector at that position, and the second is the complex part of that vector entry.

What does `DumpMachine` tell us?

```
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
```

This is the same ket we saw earlier!

$\left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

## Case Study: Generating quantum random numbers

```Q#
// build-demo.qs
namespace Build.Demo {
    operation Qrng() : Result {
        using (qubit = Qubit()) {   // Preparing 
            H(qubit);               // Operation 
            return MResetZ(qubit);  // Measure and reset 
        }
    }
}
```

We can see that this is a '.qs' file, so it is a Q# source file and it has a number of operations and functions we can import:

In [ ]:
from Build.Demo import Qrng

## Understanding `Qrng`

We can use built-in documentation as a resource.

In [ ]:
?Qrng

That tells us what we can **do** with `Qrng`:

In [ ]:
[Qrng.simulate(verbose=False) for _ in range(10)]

### Randomness brought to you by superposition!

We can use `DumpMachine` again to understand see what the `H` operation does to our qubit.

In [ ]:
Qrng.simulate(verbose=True)

### So that's nice, how about more qubits?!

## Operations with multiple qubits can create entanglement

Using Q# with Python, we can also explore other quantum development concepts, like **entanglement**.

In [ ]:
from Build.Demo import EntangleQubits
results = EntangleQubits.simulate(verbose=True)

What does `DumpRegister` tell us this time?
```
# wave function for qubits with ids (least to most significant): 0;1
0:	0.707106781186548	0
1:	0	0
2:	0	0
3:	0.707106781186548	0
```
No matter how many times we run, both measurements are equal to each other.

In [ ]:
[EntangleQubits.simulate(verbose=False) for _ in range(10)]

---

## Toy quantum algorithim: Deutsch–Jozsa 

_If I had a function that had one bit input and ouput, how many different options would I have?_

<figure style="text-align: center;">
    <img src="media/twobit.png" width="60%">
    <caption>
      <br>  
        <strong>Diagram of all possible one bit functions</strong>
    </caption>
</figure>

>#### Deutsch–Jozsa Algorithim ####
>**Problem statement:**
>
>* **GIVEN:** A black box quantum operation which takes 1 input bit and produces either a 0 or a 1 as output. We are promised that the box is either _constant_ or _balanced_. 
>					
>* **GOAL:** to determine if the box output is _constant_ or _balanced_ by evaluating sample inputs.

<figure style="text-align: center;">
    <img src="media/twobitDJ.png" width="50%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

In [ ]:
available_ops['Build.DeutschJozsa']

In [ ]:
is_zero_oracle_balanced = qsharp.compile("""
open Build.DeutschJozsa;

operation IsZeroOracleBalanced(): Bool {
    return IsOracleBalanced(ZeroOracle);
}
""")

In [ ]:
is_zero_oracle_balanced.simulate()

In [ ]:
is_not_oracle_balanced = qsharp.compile("""
open Build.DeutschJozsa;

operation IsNotOracleBalanced(): Bool {
    return IsOracleBalanced(NotOracle);
}
""")

In [ ]:
is_not_oracle_balanced.simulate()

In [ ]:
from Build.DeutschJozsa import RunDeutschJozsaAlgorithm, RunDeutschJozsaAlgorithmVerbose

In [ ]:
RunDeutschJozsaAlgorithm.simulate()

In [ ]:
RunDeutschJozsaAlgorithmVerbose.simulate()

---

## Helpful diagnostics :)

In [ ]:
for component, version in sorted(qsharp.component_versions().items(), key=lambda x: x[0]):
    print(f"{component:20}{version}")

In [ ]:
import sys
print(sys.version)